## Einleitung

In den Straßen New York Citys herrscht ein hohes Verkehrsaufkommen, wodurch Verkehrsunfälle an der Tagesordnung sind. Durch [NYC Open Data](https://opendata.cityofnewyork.us/) werden die erfassten Verkehrsunfälle für die Allgemeinheit zugänglich gemacht.

Thema dieses Berichtes und Thema unseres Projektes ist es, die öffentlich zugänglichen Unfalldaten zu nutzen, um darauf die im Abschnitt `Analyseziele` beschriebenen Analysen durchzuführen und die Ergebnisse zu visualisieren.

### Ausführungsplan

Bevor der eigentliche Bericht und der entsprechende Code genauer behandelt werden, wird hier erläutert, was installiert und beachtet werden muss, um einen möglichst reibungslosen Ablauf zu garantieren.

### Aufbau der Data Pipeline

Für das Projekt soll eine Data-Pipeline aufgebaut werden, welche die Daten in die Datenbank lädt. 
Von dort aus sollen die Daten von einem Python-Script zur Analyse angefragt werden.

Beteiligt an der Data-Pipeline ist eine Kafka-Instanz, wobei ein Python-Script einen Producer dafür darstellt und ein weiteres Python-Script einen Consumer. Das Producer-Script lädt die Daten zeilenweise aus der Datenquelle, einer CSV-Datei, in Kafka ein, während der Consumer die Daten aus Kafka ausliest und in die MongoDB schreibt. Das Analyse-Script bezieht die Daten wiederum aus der MongoDB.

### Installation der benötigten Docker-Container

Das Script `start_docker.sh` startet den MongoDB Docker Container, beziehungsweise erstellt bei Bedarf einen neuen Docker Container. Dabei wird, falls noch kein Image für die DockerDB vorhanden ist, das neueste Heruntergeladen. 

<h4 style="color:red">Das hierüber erwähnte Script existiert nicht oder?</h4>

Die weiteren benötigten Docker Container sind in der Docker-Compose-Konfiguration (`docker-compose.yml`) vorhanden.
Diese können mit `docker-compose up` erstellt oder heruntergeladen werden. 
Mit `docker-compose down` können die gestarteten Container gestoppt werden, wobei auf die Vollendigung des Befehls gewartet werden sollte.

Auf manchen Systemen befindet sich die Docker Engine nach stoppen der Docker Container mit `docker-compose down` in einem nicht-revertierbarem Fehlerzustand, wobei ein _Neustart des Computers_ unausweichlich ist. Weitere Informationen zu dem Fehler, befinden sich [auf GitHub](https://github.com/docker/for-linux/issues/162). Hier sollte gesagt werden, dass _keiner der angegebenen Workarounds_ genutzt werden sollte.

## Datenquelle

Als Datenquellen dienen die vom NYC Open Data Project bereitgestellten Unfalldaten des NYPD. Die detaillierten Unfalldaten befinden sich in 3 unterschiedlichen Datenquellen die unter Anderem als CSV-Datei verfügbar sind: [Motor Vehicle Collisions](https://data.cityofnewyork.us/browse?Data-Collection_Data-Collection=Motor+Vehicle+Collisions&q=crashes).

Enthalten in den Daten sind die am Unfall beteiligten Verkehrsteilnehmer, die Verletzten und Toten, sowie die genaue Position des Unfalls. Darüberhinaus sind noch weitere Daten in den Quellen enthalten, auf die teilweise im Analyseabschnitt genauer eingegangen wird.

## Analyseziele

Ziel der Analyse der Daten ist herauszufinden, welche saisonalen und lokalen Zusammenhänge festzustellen sind, wodurch sich besondere Hotspots im Stadtraum New Yorks lokalisieren lassen. Des weiteren soll ermittelt werden, welche Verkehrsteilnehmer besonders oft getötet oder verletzt werden, sowie an welchen Stellen sich gerade schwerwiegende Unfälle häufen. Anhand einer animierten Karte kann dadurch das Unfallgeschehen über die Zeit betrachtet werden.

<h4 style="color:red"> Kann nochmal umformuliert werden, wenn wir wissen was wir geschafft haben</h4>

## Installation der Python Packages

Damit nicht alles vollständig von uns entwickelt werden muss, wurden verschiedene Bibliotheken verwendet. Auf diese Weise kann sich die Umsetzung auf das Wesentliche beziehen.

Apache Kafka (https://kafka.apache.org/) soll dazu genutzt werden, mit Hilfe eines Producers und eines Consumers die Datensätze zeilenweise einzulesen. Dies wird durch die kafka-python Bibliothek (https://github.com/dpkp/kafka-python) stark vereinfacht.

Während der Kafka Consumer die Datensätze registiert und ausliest müssen diese ein die MongoDB eingetragen werden. Um dieses Verfahren zu vereinfachen, wird die pymongo Bibliothek (https://pypi.org/project/pymongo/) genutzt.

Es folgt die Installation der Bibliotheken über `pip install`:

In [ ]:
pip install kafka-python pymongo folium

### Importieren benötigter Module

Damit die verschiedenen Funktionen und Datentypen der Bibliotheken genutzt werden können, müssen die entsprechenden Module importiert werden. Im folgenden Abschnitt werden alle notwendigen Module, die im Laufe des Projektes genutzt werden importiert.

Zusätzlich hat sich während der Entwicklung gezeigt, dass je nach Betriebssystem Unterschiede bezüglich der Paralellisierung von Prozessen entstehen. Aus diesem Grund wird hier eine Variable `windows` deklariert, mit der im Laufe des zwischen Windows und anderen Betriebssystemen unterschieden werden kann.

Zuletzt wird hierbei die Breite des Jupyter Notebooks auf 100% gestellt, um die Lesbarkeit zu verbessern.

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
from pymongo import MongoClient
import datetime as dt
import requests
import os
from pathlib import Path
from multiprocessing import Process
from time import sleep
from data.schemas import schemas
import platform
import folium
windows = True if 'Windows' in platform.system() else False 

from IPython.core.display import display, HTML
display(HTML(
    "<style>.container { width:100% !important; }</style>"
))

DEBUG = True

## Verarbeitung der Daten

Um mit der Verarbeitung der Daten zu beginnen, müssen die Daten zunächst aufbereitet und in die MongoDB eingefügt werden. Damit mit Hilfe der MongoDB verschiedene Analysen durchgeführt werden können, sind verschiedene Schritte möglich, welche in diesem Kapitel nacheinander erläutert und durchgeführt werden.

### Herunterladen der Datensätze

Da die Daten nur verarbeitet werden können, während sie diesem Notebook zur Verfügung stehen, müssen die Daten im ersten Schritt heruntergeladen werden. Hierzu werden die Daten von der Datenquelle mithilfe des nachfolgenden Python-Scripts heruntergeladen. Resultat sind drei `.csv`-Dateien, welche die Unfalldaten zeilenweise enthalten.
Die Dateien werden im Ordner `data` gespeichert, damit sie nur einmalig heruntergeladen werden müssen. Der Ordner befindet sich im gleichen Verzeichnis wie dieses Jupyter Notebook.

In [ ]:
if not os.path.exists('data/'):
    os.mkdir('data')

for file_name, download_url in [
    ('crashes.csv', 'https://data.cityofnewyork.us/api/'
     'views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'),
    ('vehicles.csv', 'https://data.cityofnewyork.us/api/'
     'views/bm4k-52h4/rows.csv?accessType=DOWNLOAD'),
    ('persons.csv', 'https://data.cityofnewyork.us/api/'
     'views/f55k-p6yu/rows.csv?accessType=DOWNLOAD'),
]:
    if not os.path.isfile(fp:= (Path('data') / file_name)):
        with open(fp, 'wb') as crash_file:
            crash_file.write(requests.get(download_url).content)

### Produzieren der CSV-Daten in Kafka

Nachdem die Daten heruntergeladen wurden, beginnt die eigentliche Verarbeitung.
Im folgenden Codeabschnitt wird hierfür eine Funktion `send_to_kafka` definiert, welche zunächst eine Instanz des `KafkaProducer` erstellt, die dazu genutzt wird, die Daten zu produzieren. Um die Daten zu produzieren muss zeilenweise über die entsprechende `.csv` Datei iteriert werden. Nachdem eine Zeile ausgelesen wurde, wird diese in das entsprechende `Topic` produziert und muss von einem `KafkaConsumer` wieder ausgelesen werden, um die Daten nutzen zu können.

<h4 style="color:red"> Hendrik hier muss noch irgendwas mit Prozessen hin und evtl. was zur producer send zeile</h4>

In [ ]:
from pipeline_tools import send_to_kafka
                
producers = {
    'crashes': Process(
        target=send_to_kafka, args=[
            'crashes', 15000 if DEBUG else None
    ]),
    'vehicles': Process(
        target=send_to_kafka, args=[
            'vehicles', 5000 if DEBUG else None
    ]),
    'persons': Process(
        target=send_to_kafka, args=[
            'persons', 5000 if DEBUG else None
    ]),
}

### Konsumieren der produzierten Daten und Import in die Datenbank

Im nächsten Schritt wird die Funktion definiert, mit der die Daten aus Kafka ausgelesen und in die MongoDB eingefügt werden.
Hierzu wird zunächst eine Verbindung zur Datenbank aufgebaut. 

<h4 style="color:red"> Keine ahnung was dieses "as client" bedeutet und wieso man da einrücken muss aber das würde ich auch noch kurz anreißen</h4>

Sobald die Verbindung zur Datenbank bereit ist, wird ein `KafkaConsumer` erstellt, der das übergebene `topic` abonniert. Durch das Abonnieren eines Topics, erhält der Consumer zeilenweise die zuvor in das Topic gelesenen Daten. Jede Zeile wird nach dem Einlesen zunächst in ihre Bestandteile getrennt. Da es sich um Daten im CSV-Format handelt, wird die Zeile zunächst per `split(',')` bei jedem Komma getrennt.

Durch das Trennen in die einzelnen Bestandteile, sind die Daten nun bereit als Key-Value Paare in das entsprechende Dokument innerhalb der MongoDB persistiert zu werden.

<h4 style="color:red">Was genau da beim Eintragen in die DB passiert verstehe ich leider nicht aus dem Code, sollte hier aber auch erläutert werden</h4>

In [ ]:
from pipeline_tools import process_topic

consumers = {
    'vehicles': Process(
        target=process_topic, args=(
        'vehicles', schemas['vehicles']
    )),
    'persons': Process(
        target=process_topic, args=(
        'persons', schemas['persons']
    )),
    'crashes': Process(
        target=process_topic, args=(
        'crashes', schemas['crashes']
    )),
}

### Starten der Prozesse

Nachdem die Funktionen, die für das Speichern der CSV-Daten in die Datenbank zuständig sind, nun fertig definiert sind, werden diese hier aufgerufen. Da die Daten zu Personen und Fahrzeugen (Topic `nyc_persons` und `nyc_vehicles`) im Anschluss an das Persistieren in der Datenbank mit den Unfällen (`crashes`) verbunden werden müssen, werden sie vollständig verarbeitet, bevor die Daten zu den Unfällen in die Datenbank geschrieben werden. 
Dies wurde für das Windows Betriebssystem und andere Betriebssysteme unterschiedlich umgesetzt:

Windows: Da es auf Windows nicht möglich war, mehrere Prozesse innerhalb des gleichen Jupyter Notebooks zu starten (LINK???), wurden die Prozesse hier nacheinander gestartet. Durch diese Ausführung, musste der Consumer für Unfälle nur als letztes aufgerufen werden. 
<h4 style="color:red">Im Windows Teil noch ein Beweislink vielleicht?</h4>

Andere Betriebssysteme: Da mehrere Prozesse parallel laufen können, werden im folgenden Codeabschnitt zwar alle drei Producer gestartet, jedoch nur zwei (statt drei) Consumer. Der Consumer für die Unfälle wird im weiteren Verlauf dieses Berichtes dann einzeln angestoßen.

In [ ]:
if windows:
    limit = send_to_kafka('vehicles', 5000 if DEBUG else None)
    process_topic('vehicles', schemas['vehicles'], limit)
    limit = send_to_kafka('persons', 5000 if DEBUG else None)
    process_topic('persons', schemas['persons'], limit)
    limit = send_to_kafka('crashes', 15000 if DEBUG else None)
    process_topic('crashes', schemas['crashes'], limit)
else:
    for topic in 'persons vehicles crashes'.split(' '):
        producers[topic].start()
    sleep(2)
    for topic in 'persons vehicles'.split(' '):
        consumers[topic].start()

### Starten des Join-Prozesses (außer auf Windows)
Im letzten Schritt muss der Prozess gestartet werden, der das Verbinden der beiden Tabellen `vehicles` und `persons` in die Daten des Topics `nyc_crashes` durchführt. Dieser Schritt wurde auf Windows bereits im vorherigen Codeabschnitt erledigt, da dieser auf Windows nicht paralellisiert wird. 

Für alle anderen Betriebssysteme ist es wichtig, dass die parallelisierten Prozesse zum Produzieren aller Daten und zum Einlesen der `nyc_vehicles` und `nyc_persons` Topics beendet sind. Die Ausführung des vorherigen Codeblocks sollte also abgeschlossen sein. Damit die folgende Funktion bei einem `Kernel` → `Restart & Run All` nicht zu früh ausgeführt wird, zählt der nächste Codeblock als ein Breakpoint.

In [ ]:
if not windows:
    assert False, 'breakpoint'

Diese Zelle muss auf Windows nach der Beendigung der oben erwähnten Zelle ausgeführt werden.

In [ ]:
if not windows:
    consumers['crashes'].start()

In [ ]:
from pipeline_tools import aggregate_data

aggregate_data()

# Datenanalyse

## Unfallkarte

### Jahr 2014

In [ ]:
from folium.plugins import HeatMap
data = []
layer_conf = {'zoom_start': 10.5, 'min_zoom': 10, 'max_zoom': 14}

base_map = folium.Map(location=[40.73, -73.8], **layer_conf, tiles='Stamen Water Color')

#folium.TileLayer('Stamen Water Color', **layer_conf).add_to(base_map)
#folium.LayerControl().add_to(base_map)

with MongoClient('mongodb://localhost:27017') as client:
    #crashes_by_year = client['nyc_crashes']['crashes_by_year']
    
    #year_2014 = crashes_by_year.find_one({'_id': {'$eq': str(2014)}})
    for idx, crash in enumerate(client['nyc_crashes']['crashes'].find()):
        if idx > 500:
            break
        lon, lat = crash['longitude'], crash['latitude']
        if lon and lat:
            data.append([lat, lon])
HeatMap(data=data, radius=15).add_to(base_map)


base_map

In [ ]:

folium.TileLayer('Stamen Terrain', **layer_conf).add_to(m)
folium.TileLayer('Stamen Toner', **layer_conf).add_to(m)
folium.TileLayer('Stamen Water Color', **layer_conf).add_to(m)
folium.LayerControl().add_to(m)

data = []

with MongoClient('mongodb://localhost:27017') as client:
    #crashes_by_year = client['nyc_crashes']['crashes_by_year']
    
    #year_2014 = crashes_by_year.find_one({'_id': {'$eq': str(2014)}})
    for idx, crash in enumerate(client['nyc_crashes']['crashes'].find()):
        if idx > 500:
            break
        lon, lat = crash['longitude'], crash['latitude']
        if lon and lat:
            data.append([lat, lon])
            continue
            folium.Circle(
                radius=1000,
                location=[lat, lon],
                popup=crash['crash_date'],
                fill_color='red',
                fill=True,
                color='#00000000'
            ).add_to(m)
m = HeatMap(data=data)

In [ ]:
m.render()

# Ergebnis